<a href="https://colab.research.google.com/github/monsterhunters/Stable-Diffusion/blob/main/New_SD_V1_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os, subprocess, time, glob

output_path = 'AI_PICS' #@param {type:"string"}
#@markdown Save images, settings, Lora models, embeddings, load models in Google Drive.

from google.colab import drive
drive.mount('/content/drive')

Save_In_Google_Drive = 'Nothing' #@param ['Everything', 'Small models, images and settings (Recommended)', "Nothing"]
class GDriveSaveMode:
    Everything = 1
    Models_only = 2
    Nothing = 3
if Save_In_Google_Drive == 'Small models, images and settings (Recommended)':
  gMode = GDriveSaveMode.Models_only
elif Save_In_Google_Drive == 'Everything':
  gMode = GDriveSaveMode.Everything
elif Save_In_Google_Drive == 'Nothing':
  gMode = GDriveSaveMode.Nothing
else:
  raise ValueError("Save_In_Google_Drive value invalid.")

Clear_Log = False #@param{type:'boolean'}
#@markdown ## Only check the models you are going to use:
###@markdown ### v1.5 models:
#v1_5_model = False #@param{type: "boolean"}
#v1_5_inpainting_model = False #@param{type: "boolean"}
#F222_model = False #@param{type: "boolean"}
#Realistic_Vision_model = False #@param{type: "boolean"}
#Realistic_Vision_Inpainting_model = False #@param{type: "boolean"}
#DreamShaper_model = False #@param{type: "boolean"}
#DreamShaper_Inpainting_model = False #@param{type: "boolean"}
#OpenJourney_model = False #@param{type: "boolean"}
#Anything_v3_model = False #@param{type: "boolean"}
#Inkpunk_Diffusion_model = False #@param{type: "boolean"}
#instruct_pix2pix_model = False #@param{type: "boolean"}

###@markdown ### v2 models:
#v2_1_768_model = False #@param{type: "boolean"}
#v2_1_512_model = False #@param{type: "boolean"}
#v2_depth_model = False #@param{type: "boolean"}

#@markdown ### SDXL models
SDXL_1 = False #@param{type: "boolean"}


#@markdown ### Extensions:
ControlNet = False #@param{type: "boolean"}
Deforum = False #@param{type: "boolean"}
Regional_Prompter = False #@param{type: "boolean"}
Ultimate_SD_Upscale = False #@param{type: "boolean"}
Openpose_Editor = False #@param{type: "boolean"}
ADetailer = False #@param{type: "boolean"}
AnimateDiff = False #@param{type: "boolean"}
text2video = False #@param{type: "boolean"}
Reactor = False #@param{type: "boolean"}



###@markdown ### Install models from URL (Hugging Face link only) (separate them with comma).
##Model_from_URL = '' #@param {type: "string"}
##Save_a_copy_in_Google_Drive = False #@param{type: "boolean"}
###@markdown (Saved models will be available next time you start)

###@markdown ### Install extensions from URL (separate them with comma).
##Extensions_from_URL = '' #@param {type: "string"}

###@markdown ### Extra Web-UI arguments
##Extra_arguments = '--disable-model-loading-ram-optimization --opt-sdp-no-mem-attention' #@param {type: "string"}


if gMode != GDriveSaveMode.Nothing:
  # connect to google drive
  from google.colab import drive
  drive.mount('/content/drive')
  output_path = '/content/drive/MyDrive/' + output_path
  if gMode == GDriveSaveMode.Everything:
    # install A1111 in AI_PIC
    root = output_path
  elif gMode == GDriveSaveMode.Models_only:
    # install A1111 in Colab
    root = '/content/'
    # make necessary directories if not exists
    !mkdir -p {output_path}/outputs
    !mkdir -p {output_path}/models
    !mkdir -p {output_path}/ESRGAN
    !mkdir -p {output_path}/hypernetworks
  else:
    raise ValueError("Unexpected gMode: %s"%gMode)
else:
  # Don't connect to google drive
  output_path = '/content/' + output_path
  root = '/content/'

!mkdir -p {root}
os.chdir(root)
!apt-get -y install -qq aria2
!pip install pyngrok
!git clone https://github.com/monsterhunters/stablecode.git


def clear():
    from IPython.display import clear_output; return clear_output()

def fetch_bytes(url_or_path):
    if str(url_or_path).startswith('http://') or str(url_or_path).startswith('https://'):
        from urllib.request import urlopen
        return urlopen(url_or_path)
    return open(url_or_path, 'r')

def packages():
    import sys, subprocess
    return [r.decode().split('==')[0] for r in subprocess.check_output([sys.executable, '-m', 'pip', 'freeze']).split()]

def downloadModel(url):
  if 'huggingface.co' in url:
    filename = url.split('/')[-1]
    filename = filename.removesuffix('?download=true')
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {url}  -o {filename}
  else:
    # civitai
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {url}


def download_models():
  %cd {root}/stablecode/models/Stable-diffusion
  print('⏳ Downloading models ...')
  if v1_5_model:
    downloadModel('https://huggingface.co/ckpt/sd15/resolve/main/v1-5-pruned-emaonly.ckpt')
  if v1_5_inpainting_model:
    downloadModel('https://huggingface.co/runwayml/stable-diffusion-inpainting/resolve/main/sd-v1-5-inpainting.ckpt')
  if F222_model:
    downloadModel('https://huggingface.co/acheong08/f222/resolve/main/f222.ckpt')

  if Realistic_Vision_model:
    downloadModel('https://huggingface.co/SG161222/Realistic_Vision_V5.1_noVAE/resolve/main/Realistic_Vision_V5.1_fp16-no-ema.safetensors')
  if Realistic_Vision_Inpainting_model:
    downloadModel('https://huggingface.co/SG161222/Realistic_Vision_V5.1_noVAE/resolve/main/Realistic_Vision_V5.1-inpainting.safetensors')

  if DreamShaper_model:
    downloadModel('https://huggingface.co/Lykon/DreamShaper/resolve/main/DreamShaper_8_pruned.safetensors')
  if DreamShaper_Inpainting_model:
    downloadModel('https://huggingface.co/Lykon/DreamShaper/resolve/main/DreamShaper_8_INPAINTING.inpainting.safetensors')

  if OpenJourney_model:
    downloadModel('https://huggingface.co/prompthero/openjourney/resolve/main/mdjrny-v4.ckpt')
  if Anything_v3_model:
    downloadModel('https://huggingface.co/Linaqruf/anything-v3.0/resolve/main/anything-v3-fp16-pruned.safetensors')
  if Inkpunk_Diffusion_model:
    downloadModel('https://huggingface.co/Envvi/Inkpunk-Diffusion/resolve/main/Inkpunk-Diffusion-v2.ckpt')
  if instruct_pix2pix_model:
    downloadModel('https://huggingface.co/timbrooks/instruct-pix2pix/resolve/main/instruct-pix2pix-00-22000.ckpt')

  if v2_1_768_model:
    downloadModel('https://huggingface.co/stabilityai/stable-diffusion-2-1/resolve/main/v2-1_768-ema-pruned.ckpt')
    !wget -nc https://raw.githubusercontent.com/Stability-AI/stablediffusion/main/configs/stable-diffusion/v2-inference-v.yaml -O {root}/stablecode/models/Stable-diffusion/v2-1_768-ema-pruned.yaml

  if v2_1_512_model:
    downloadModel('https://huggingface.co/stabilityai/stable-diffusion-2-1-base/resolve/main/v2-1_512-ema-pruned.ckpt')
    !wget -nc https://raw.githubusercontent.com/Stability-AI/stablediffusion/main/configs/stable-diffusion/v2-inference.yaml -O {root}/stablecode/models/Stable-diffusion/v2-1_512-ema-pruned.yaml

  if v2_depth_model:
    downloadModel('https://huggingface.co/stabilityai/stable-diffusion-2-depth/resolve/main/512-depth-ema.ckpt')
    !wget -nc https://raw.githubusercontent.com/Stability-AI/stablediffusion/main/configs/stable-diffusion/v2-midas-inference.yaml -O {root}/stablecode/models/Stable-diffusion/512-depth-ema.yaml

  if SDXL_1:
    downloadModel('https://huggingface.co/stabilityai/stable-diffusion-xl-base-1.0/resolve/main/sd_xl_base_1.0.safetensors')
    downloadModel('https://huggingface.co/stabilityai/stable-diffusion-xl-refiner-1.0/resolve/main/sd_xl_refiner_1.0.safetensors')

  if Model_from_URL:
      for m in Model_from_URL.split(','):
        %cd {root}/stablecode/models/Stable-diffusion
        downloadModel(m)
        if Save_a_copy_in_Google_Drive and gMode == GDriveSaveMode.Models_only:
          %cd {output_path}/models
          downloadModel(m)

  # download VAEs
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/stabilityai/sd-vae-ft-ema-original/resolve/main/vae-ft-ema-560000-ema-pruned.ckpt -d {root}/stablecode/models/VAE/ -o vae-ft-ema-560000-ema-pruned.ckpt
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/stabilityai/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.ckpt -d {root}/stablecode/models/VAE/ -o vae-ft-mse-840000-ema-pruned.ckpt

def installxformers():
  %pip install -q xformers

def updatePython():
  !python --version > /content/pyversion
  with open('/content/pyversion', 'r') as file:
      if '3.10' in file.read():
        print('Already python 3.10. Skip install.')
        return

  #install python 3.10
  !apt-get update -y
  !apt-get install python3.10

  #change alternatives
  !rm /usr/local/bin/python
  !rm /usr/local/bin/pip
  !sudo apt-get install python3.10-distutils
  !sudo update-alternatives --install /usr/local/bin/python python /usr/bin/python3.10 2
  !wget https://bootstrap.pypa.io/get-pip.py && python get-pip.py



def installControlNet():
  print("Installing ControlNet extension...")
  %cd {root}/stablecode/extensions
  !git clone https://github.com/monsterhunters/codecontrolnet.git
  %cd {root}/stablecode/extensions/codecontrolnet
  !pip install -r requirements.txt

  %cd {root}/stablecode/extensions/codecontrolnet/models
  downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11e_sd15_ip2p.pth')
  downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11e_sd15_shuffle.pth')
  downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11f1e_sd15_tile.pth')
  downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11f1p_sd15_depth.pth')
  downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_canny.pth')
  downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_inpaint.pth')
  downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_lineart.pth')
  downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_mlsd.pth')
  downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_normalbae.pth')
  downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_openpose.pth')
  downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_scribble.pth')
  downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_seg.pth')
  downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_softedge.pth')
  downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15s2_lineart_anime.pth')
  downloadModel('https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_color_sd14v1.pth')
  downloadModel('https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_style_sd14v1.pth')


  print('Install QR code models...')
  downloadModel('https://huggingface.co/monster-labs/control_v1p_sd15_qrcode_monster/resolve/main/control_v1p_sd15_qrcode_monster.safetensors')
  downloadModel('https://huggingface.co/monster-labs/control_v1p_sd15_qrcode_monster/resolve/main/v2/control_v1p_sd15_qrcode_monster_v2.safetensors')
  downloadModel('https://huggingface.co/Nacholmo/controlnet-qr-pattern-sdxl/resolve/main/automatic1111/control_v01u_sdxl_qrpattern.safetensors')


  print("Install ControlNet SDXL models...")
  downloadModel('https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/diffusers_xl_canny_full.safetensors')
  downloadModel('https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/diffusers_xl_depth_mid.safetensors')
  downloadModel('https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/ip-adapter_xl.pth')
  downloadModel('https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/kohya_controllllite_xl_blur.safetensors')
  downloadModel('https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/kohya_controllllite_xl_blur_anime.safetensors')
  downloadModel('https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/kohya_controllllite_xl_scribble_anime.safetensors')
  downloadModel('https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/sai_xl_recolor_256lora.safetensors')
  downloadModel('https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/sai_xl_sketch_256lora.safetensors')
  downloadModel('https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/t2i-adapter_diffusers_xl_lineart.safetensors')

def clear():
    from IPython.display import clear_output; return clear_output()


def installADetailer():
  %cd {root}/stablecode/extensions
  !git clone https://github.com/Bing-su/adetailer

def installReactor():
  %cd {root}/stablecode/extensions
  !git clone https://github.com/monsterhunters/codemaster-roop.git

def installUltimateSDUpscale():
  %cd {root}/stablecode/extensions
  !git clone https://github.com/Coyote-A/ultimate-upscale-for-automatic1111

def installOpenPoseEditor():
  %cd {root}/stablecode/extensions
  !git clone https://github.com/fkunn1326/openpose-editor

def installDeforum():
  !git clone https://github.com/deforum-art/deforum-for-automatic1111-webui {root}/stablecode/extensions/deforum
  #!cd {root}/stablecode/extensions/deforum; git checkout c42834645805e0f26172888b29f5a9210063db14
  !apt-get install libvulkan1

def installRegionalPrompter():
  !git clone https://github.com/monsterhunters/codemaster-regional-prompter.git {root}/stablecode/extensions/codemaster-regional-prompter

def installtext2video():
  %cd {root}/stablecode/extensions
  !git clone https://github.com/monsterhunters/codemaster-text2video.git
  %mkdir -p {root}/stablecode/models/text2video/t2v
  %cd {root}/stablecode/models/text2video/t2v
  downloadModel('https://huggingface.co/damo-vilab/modelscope-damo-text-to-video-synthesis/resolve/main/VQGAN_autoencoder.pth')
  downloadModel('https://huggingface.co/damo-vilab/modelscope-damo-text-to-video-synthesis/resolve/main/configuration.json')
  downloadModel('https://huggingface.co/damo-vilab/modelscope-damo-text-to-video-synthesis/resolve/main/open_clip_pytorch_model.bin')
  downloadModel('https://huggingface.co/damo-vilab/modelscope-damo-text-to-video-synthesis/resolve/main/text2video_pytorch_model.pth')

def installAnimateDiff():
  %cd {root}/stablecode/extensions
  !git clone https://github.com/monsterhunters/codemaster-animatediff.git
  # !git checkout b192a2551a5ed66d4a3ce58d5d19a8872abc87ca
  %cd {root}/stablecode/extensions/codemaster-animatediff/model
  downloadModel('https://huggingface.co/guoyww/animatediff/resolve/main/mm_sd_v14.ckpt')
  downloadModel('https://huggingface.co/guoyww/animatediff/resolve/main/mm_sd_v15.ckpt')
  downloadModel('https://huggingface.co/guoyww/animatediff/resolve/main/mm_sd_v15_v2.ckpt')

if gMode == GDriveSaveMode.Everything:
  # delete existing repositories and reclone so the file mode fix can be applied
  # otherwise some will only be cloned in the final launch, causing some to fail to checkout.
  deleteRepos()
  cloneRepositories()
  applyGitFilemode()

if ControlNet:
  installControlNet()

if Deforum:
  installControlNet() # Deforum needs controlnet to work now
  installDeforum()

if Regional_Prompter:
  installRegionalPrompter()

if Ultimate_SD_Upscale:
  installUltimateSDUpscale()

if Openpose_Editor:
  installOpenPoseEditor()

if ADetailer:
  installADetailer()

if AnimateDiff:
  installAnimateDiff()

if text2video:
  installtext2video()

if Reactor:
  installReactor()

# clear output
if Clear_Log:
  clear()


def applyGitFilemode():
  # git default file mode prevents checkout and fail
  print('Apply git filemode false')
  !cd {root}/stablecode/repositories/k-diffusion;git config core.filemode false
  !cd {root}/stablecode/repositories/stable-diffusion-stability-ai;git config core.filemode false
  !cd {root}/stablecode/repositories/taming-transformers;git config core.filemode false
  !cd {root}/stablecode/repositories/CodeFormer;git config core.filemode false
  !cd {root}/stablecode/repositories/BLIP;git config core.filemode false


def cloneRepositories():
  !git clone https://github.com/crowsonkb/k-diffusion.git {root}/stablecode/repositories/k-diffusion
  !git clone https://github.com/Stability-AI/stablediffusion.git {root}/stablecode/repositories/stable-diffusion-stability-ai
  !git clone https://github.com/CompVis/taming-transformers.git {root}/stablecode/repositories/taming-transformers
  !git clone https://github.com/sczhou/CodeFormer.git {root}/stablecode/repositories/CodeFormer
  !git clone https://github.com/salesforce/BLIP.git {root}/stablecode/repositories/BLIP

def installExtensionsFromURL(urls):
  %cd {root}/stablecode/extensions
  for url in urls.split(','):
    print("Cloning extension from URL: %s"%url)
    !git clone {url}

def lowRamPatch():
  print('Apply lowram patch')
  !sed -i 's/dict()))$/dict())).cuda()/g'  {root}/stablecode/repositories/stable-diffusion-stability-ai/ldm/util.py

def searchAndReplace(filePath, orignalStr, newStr):
  orignalStr = orignalStr.replace('/', '\/')
  newStr = newStr.replace('/', '\/')
  !sed -i 's/{orignalStr}/{newStr}/g' {filePath}


def deleteRepos():
  # delete repository directories in webui
  !rm -rf {root}/stablecode/repositories

updatePython()

#if  gMode != GDriveSaveMode.Everything: //Need to study
#   lowRamPatch()

%cd /content/stablecode

In [8]:
#@title RUN
!npm install -g localtunnel

import subprocess
import threading
import time
import socket
import urllib.request

def iframe_thread(port):
  while True:
      time.sleep(0.5)
      sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
      result = sock.connect_ex(('127.0.0.1', port))
      if result == 0:
        break
      sock.close()
  print("\nComUI finished loading, trying to launch localtunnel (if it gets stuck here localtunnel is having issues)\n")

  print("The password/enpoint ip for localtunnel is:", urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))
  p = subprocess.Popen(["lt", "--port", "{}".format(port)], stdout=subprocess.PIPE)
  for line in p.stdout:
    print(line.decode(), end='')


threading.Thread(target=iframe_thread, daemon=True, args=(7860,)).start()

!python launch.py --disable-model-loading-ram-optimization --opt-sdp-no-mem-attention --ckpt-dir "/content/drive/Shareddrives/BSSG-RCN-(70)/HTZ/CodeUI/models/checkpoints" --vae-dir "/content/drive/Shareddrives/BSSG-RCN-(70)/HTZ/CodeUI/models/vae" --embeddings-dir "/content/drive/Shareddrives/BSSG-RCN-(70)/HTZ/CodeUI/models/embeddings" --lora-dir "/content/drive/Shareddrives/BSSG-RCN-(70)/HTZ/CodeUI/models/loras" --esrgan-models-path "/content/drive/Shareddrives/BSSG-RCN-(70)/HTZ/CodeUI/models/upscale_models"